In [1]:
import cv2

In [2]:
def find_large_label(img, gray, img_bin,bin_inverse = True ):

    _, labels, stats, centroids = cv2.connectedComponentsWithStats(img_bin)
    stats = sorted(stats, key = lambda x: x[4],reverse=True)
    # for i, rec in enumerate(stats[1:2]):
    #     x,y,w,h, area = rec
    #     cv2.rectangle(img, (x,y,w,h), (0,0,255))

    # cv2.imshow('item_img', img)
    # cv2.imshow('bin', img_bin)
    # cv2.imshow('item_bin', img_bin)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    # print(stats)
    if bin_inverse == True:
        return stats[1]
    else:
        return stats[0]

In [3]:
def get_threshold(img,gray,bin_inverse = True):
    if bin_inverse:
        _, img_bin = cv2.threshold(gray, -1,255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
    else:
        _, img_bin = cv2.threshold(gray, -1,255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    return img_bin


In [4]:
def preproces(img, bin_inverse = True, only_bin = False):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_bin = get_threshold(img,gray,bin_inverse = True)

    large_stat = find_large_label(img, gray, img_bin)
    try:
        item_img = img[large_stat[1]:large_stat[1]+large_stat[3],large_stat[0]:large_stat[0]+large_stat[2], :]
        item_gray = gray[large_stat[1]:large_stat[1]+large_stat[3],large_stat[0]:large_stat[0]+large_stat[2]]
    except:
        item_img, item_gray= img,gray

    item_bin = get_threshold(item_img,item_gray,bin_inverse = False)
    return  item_img, item_gray, item_bin

In [5]:
img_ok = cv2.imread('../team/images/true_ok/GSY827AN7A1356_AAO11960K_PKT10_CM1EQSUA0012_20220711210457_DirectLight_OK.jpg')
img_ng = cv2.imread('../team/images/true_ng/GSY827AN7B0519_AAO12705K_PKT08_CM1EQSUA0011_20220711213213_DirectLight_NG.jpg')
img_ng2 = cv2.imread('../team/images/true_ng/GSY827AN7A1641_AAO19043K_PKT13_CM1EQSUA0012_20220711220653_DirectLight_NG.jpg')
img_over = cv2.imread('../team/images/overkill/GSY827AN7B0355_AAO04537K_PKT01_CM1EQSUA0011_20220711234650_DirectLight_NG.jpg')


In [15]:
def find_contours(img, key = False):
    img = cv2.resize(img, (0,0), fx = 0.3, fy = 0.3, interpolation=cv2.INTER_CUBIC)
    item_img, item_gray, item_bin = preproces(img)
    
    contour, hierachy = cv2.findContours(item_bin, cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)

    for i, cnt in enumerate(contour):
        if cv2.contourArea(cnt) > 180000:
            cv2.drawContours(item_img, [cnt], 0,(0,0,255),5)
    cv2.imshow('img', img)
    cv2.imshow('item_img', item_img)
    # cv2.imshow('bin', item_bin)
    if key:
        key_val = cv2.waitKey(0)
        cv2.destroyAllWindows()
        return key_val
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # return hierachy



In [16]:
find_contours(img_ok)
find_contours(img_ng)
find_contours(img_ng2)

In [17]:
find_contours(img_over)

In [20]:
import os

def check_img(kind = 'overkill'):
    file_path = '../team/images/'+kind+'/'
    img_paths = os.listdir(file_path)
    for i in range(len(img_paths)):
        img_over = cv2.imread(file_path+img_paths[i])
        key_val = find_contours(img_over, key = True)
        if key_val == ord('q'):
            break

In [21]:
check_img()